In [18]:
from unidecode import unidecode

In [76]:
from pycurl import Curl
from io import BytesIO
import re

def download_cheat(util):
    # Create a pycUrl instance
    c = Curl()
    buffer = BytesIO()

    c.setopt(c.URL, f'cht.sh/{util}')
    c.setopt(c.WRITEDATA, buffer)

    # Make the request
    c.perform()

    # Close the connection
    c.close()

    body = buffer.getvalue()

    ansi_escape_8bit = re.compile(
        br'(?:\x1B[@-Z\\-_]|[\x80-\x9A\x9C-\x9F]|(?:\x1B\[|\x9B)[0-?]*[ -/]*[@-~])'
    )
    return ansi_escape_8bit.sub(b'', body).decode('ascii', errors='ignore')

def get_cheat(util):
    full_text = download_cheat(util)
    if 'tldr:' in full_text:
        full_text = re.findall(r'tldr:[\w-]*?\s\n(.+)', full_text, re.DOTALL)[0]
    
    full_text = full_text.replace('\n\n', '\n')
    
    full_text = re.sub(r'<((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*>', '', full_text)
    
    return full_text

In [63]:
import yaml
from tqdm.auto import tqdm

In [60]:
with open('tools_docs.yml') as f:
    all_tools = yaml.safe_load(f)

In [64]:
tool_cheatsheets = {}

In [79]:
for tool in tqdm(list(all_tools.keys())):
    try:
        tool_cheatsheets[tool] = get_cheat(tool)
    except:
        print(tool, 'got exception!')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 119/119 [00:10<00:00, 10.88it/s]


In [83]:
with open('tools_cheatsheets.yml', 'w') as f:
    yaml.safe_dump(tool_cheatsheets, f)

In [81]:
tool_cheatsheets['rev']

'# rev\n# Reverse a line of text.\n# More information: .\n# Reverse the text string "hello":\necho "hello" | rev\n# Reverse an entire file and print to `stdout`:\nrev path/to/file\n'

In [82]:
tool_cheatsheets['ssh-keygen']

'# ssh-keygen\n# Generate ssh keys used for authentication, password-less logins, and other things.\n# More information: .\n# Generate a key interactively:\nssh-keygen\n# Specify file in which to save the key:\nssh-keygen -f ~/.ssh/filename\n# Generate an ed25519 key with 100 key derivation function rounds:\nssh-keygen -t ed25519 -a 100\n# Generate an RSA 4096-bit key with email as a comment:\nssh-keygen -t dsa|ecdsa|ed25519|rsa -b 4096 -C "comment|email"\n# Remove the keys of a host from the known_hosts file (useful when a known host has a new key):\nssh-keygen -R remote_host\n# Retrieve the fingerprint of a key in MD5 Hex:\nssh-keygen -l -E md5 -f ~/.ssh/filename\n# Change the password of a key:\nssh-keygen -p -f ~/.ssh/filename\n# Change the type of the key format (for example from OPENSSH format to PEM), the file will be rewritten in-place:\nssh-keygen -p -N "" -m PEM -f ~/.ssh/OpenSSH_private_key\n'